# FUNSD Data Load

Dieses Notebook lädt den FUNSD-Datensatz über HuggingFace Datasets und zeigt die wichtigsten Felder für LayoutLM.

In [ ]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
funsd = load_dataset("nielsr/funsd")

/Users/annakoernig/multimodal-document-ai-evaluation/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DatasetNotFoundError: Dataset 'funsd' doesn't exist on the Hub or cannot be accessed

Hinweis: Falls der Hub-Zugriff fehlschlägt, stelle sicher, dass du bei Hugging Face eingeloggt bist (`huggingface-cli login`). 
Wenn du den Datensatz lokal hast, kannst du ihn alternativ mit `load_dataset` und lokalen Dateien laden.

In [ ]:
# Splits anzeigen
list(funsd.keys())
for split_name, split in funsd.items():
    print(split_name, split, "n=", len(split))

In [ ]:
# Einzelnes Beispiel
example = funsd["train"][0]
example

## Relevante Felder für LayoutLM

* **tokens/words**: Die OCR-Token (Wortstücke), die LayoutLM als Eingabe verarbeitet (je nach Dataset-Version heißen sie `tokens` oder `words`).
* **bboxes**: Bounding Boxes pro Token (normalisiert auf 0–1000), damit LayoutLM Layout-Informationen nutzen kann.
* **ner_tags**: Token-Labels für das Token-Classification-Setup (z. B. Entitäten in Formularen).

In [ ]:
# Minimaler Sanity Check (CPU-friendly)
required_fields = {"bboxes", "ner_tags"}
for split_name, split in funsd.items():
    sample = split[0]
    token_field = ("tokens" if "tokens" in sample else "words" if "words" in sample else None)
    assert token_field, f"Missing token field (tokens/words) in {split_name}: {sample.keys()}"
    missing = required_fields - sample.keys()
    assert not missing, f"Missing fields in {split_name}: {missing}"
    tokens = sample[token_field]
    assert len(tokens) == len(sample["bboxes"]) == len(sample["ner_tags"]), (
        f"Length mismatch in {split_name}: "
        f"{token_field}={len(tokens)}, bboxes={len(sample['bboxes'])}, ner_tags={len(sample['ner_tags'])}"
    )

"Sanity check passed."